## Advanced Python - Notes

### The Python Data Model
The first thing one should know when understanding advanced concepts in Python is that every item in Python is an object, which functions as an abstraction for data, encapsulating state (data) and behavior (methods). On top of that, these objects are characterized by having an `identity`, a `type` and a `value`. This characterization is important because it allows Python to manage an object's lifecycle throughout program execution. Let's dive a little deeper into how Python uses these items to manage its objects.

#### ID
In Python, an object's identity never changes once it has been created. This identity is the memory address of the object. In this scenario, the `is` operator is the most suitable way to compare the identity of two objects instead of `==`.  It is worth mentioning that the *id* sometimes is obtained through some optimizations, as listed below:

In [21]:
first_value = 5
print(id(first_value))

second_value = 2
print(id(second_value))

third_value = 2
print(id(third_value))

print(first_value is second_value)
print(second_value is third_value)

139683335127408
139683335127312
139683335127312
False
True


The pattern observed above only refer to numbers between -5 and 256. This is because Python stores small integers in a cache due to performance reasons. In those cases, Python pre-allocates the integer objects created and reuses them as an attempt to save memory.
When two variables are created and they have the same value, Python pre-allocates this value ensuring that both variables refers to the same object in memory. Whereas for values outside this range, Python allocates a new object in memory each time that an integer variable is created, as seen below:

In [2]:
first_value = 256
print(id(first_value))

second_value = 256
print(id(second_value))

third_value = -5
print(id(third_value))

fourth_value = -5
print(id(fourth_value))

print(first_value is second_value)
print(third_value is fourth_value)

print(15 * "-")

first_value = 257
print(id(first_value))

second_value = 257
print(id(second_value))

third_value = -6
print(id(third_value))

fourth_value = -6
print(id(fourth_value))

print(first_value is second_value)
print(third_value is fourth_value)

2817949078032
2817949078064
2817949077968
2817949078352
False
False


This behavior extend in certain level to strings and do not is extended to floats.

In [7]:
first_value = 2.56
print(id(first_value))

second_value = 2.56
print(id(second_value))

third_value = -5.0
print(id(third_value))

fourth_value = -5.0
print(id(fourth_value))

print(first_value is second_value)
print(third_value is fourth_value)

2817949077392
2817949077360
2817949076880
2817949077328
False
False


In [11]:
first_value = "Hello_world"
print(id(first_value))

second_value = "Hello_world"
print(id(second_value))

third_value = "Hello, world!"
print(id(third_value))

fourth_value = "Hello, world!"
print(id(fourth_value))

print(first_value is second_value)
print(third_value is fourth_value)

140002182303664
140002182303664
140002182303280
140002182307952
True
False


For strings, Python performs string interning selectively. This means that string literals that look like identifiers are typically interned automatically. Interning is a type of caching that applies to specific object instances and refers to Python reusing the string object instead of creating a new one every time the same string literal is used.

In a broader sense, Python can perform some optimizations with immutable objects that may seem to have the same interning performed with integers and strings, but that are actually compiler optimizations. In these optimizations, the interpreter understands that some immutable objects are the same if they are placed in different parts of the code:

In [5]:
first_value = (1, 2, 3)
print(id(first_value))

second_value = (1, 2, 3)
print(id(second_value))

third_value = {1, 2, 3}
print(id(third_value))

fourth_value = {1, 2, 3}
print(id(fourth_value))

print(first_value is second_value)
print(third_value is fourth_value)


# Optimizations occurs here:
def test_id():
    a = (1, 2, 3)
    b = (1, 2, 3)

    print(id(a))
    print(id(b))

    print(f"Is a equals b? The answer is: {a is b}")


test_id()

140568893257856
140568892681856
140568892797920
140568892800160
False
False
140568893131712
140568893131712
Is a equals b? The answer is: True


The above optimization occurs only in the function namespace due to its restrictive nature. The function namespace is a confined and well-defined scope of code analysis. Therefore, allowing the compiler to make some optimizations more aggressive when compared to its application at the global namespace (which is more unpredictable and dynamic).  

The *id* function has some uses that are related to scenarios where the the management of object identities is necessary (e.g debbuging). These situations generally involve identifying duplicates, performing caching/memoization, and implementing low-level optimizations. Although it has its uses, the *id* function is rarely applied in Python codebases and can be replaced in most cases by using the `is` operator.

### Type  
The object's type defines which operations can be applied to the object itself. It also defines the possible values that an object of the referred type can have. We can check the type of an object using the  `type()` function:

In [4]:
a = 5
print(type(a))
b = [1, 2, 3]
print(type(b))
print(type(type))
print(type(object))

<class 'int'>
<class 'list'>
<class 'type'>
<class 'type'>


The `type` object is a metaclass (a class of a class) and also a function that returns the type of an object. In this sense, `type` is an instance of itself, forming a recursive loop and is considered the foundational class of the Python language, because it is the class from which all other classes are derived, directly or indirectly. 

This self-referential characteristic of the `type` metaclass is what allows it to act as a foundation for all classes. Python allows us to create metaclasses for our classes and these metaclasses are also derived from the `type` class.  

It is important to note that `type` is the default metaclass in Python which defines how a class behaves, allowing the customization of class creation, modification, and deletion. Furthermore, it also can be thought of as classes of classes (a class is an instance of its metaclass). In this scenario, every class in Python is an instance of `type`, even itself.  

Another important concept in Python is base classes. This category of objects forms the classes that other classes inherit from. In Python, the `object` class is the one inherited by all new-style classes (with all its attributes and methods).  

Besides getting the type of an object, the `type` function can also be used to create a class dynamically. This feature has some interesting applications, as demonstrated below:  

In [1]:
MyDynamicClass = type(
    "MyDynamicClass",  # Name
    (object,),  # Bases
    {"x": 5, "y": 10, "z": staticmethod(lambda x: x * x)},  # Dict
)
print(MyDynamicClass.x)
print(MyDynamicClass.z(5))

instance = MyDynamicClass()

print(instance.x)
print(instance.z(5))

5
25
5
25


The use of dynamically created classes follows a clear pattern that is mostly suited to situations when the properties, methods, or base classes of the Python's objects are determined by external factors only known at runtime, such as user input, configuration data, or external schemas. This approach fits properly in scenarios where static class creation would lead to excessive code duplication or complexity.  
A helpful way of thinking in terms of using this approach is to evaluate if the behavior of a class is dependent on conditions only known at runtime, requires class dynamic generation/modification, aims to avoid code duplication, or allows end users to modify/extend the system without altering the core codebase.   
Although very useful in certain situations, the use of type in this manner adds a significant level of complexity to the code, requiring its use to always be well-considered and accounting for possible trade-offs.   

There are some use cases that cover the vast majority of situations where creating a class dynamically could be helpful. One such use case is when there's a requirement to create a class at runtime that should contain type hints:

In [2]:
from typing import Type, cast


class BaseClass:
    pass


# The cast function cast a value to a type, returning it unchanged. It is only useful for type-checkers
# because it signals that the type has the designated value.
cast(Type[BaseClass], type("SubClass", (BaseClass,), {}))

__main__.SubClass

It is also possible to dynamically create a class with slots:

In [3]:
MyClass = type("MyClass", (), {"__slots__": ("a", "b")})
print(MyClass.a)

<member 'a' of 'MyClass' objects>


Slots allow for the explicit specification of which instance attributes an object instances are expected to have. This results in faster attribute access and reduced memory usage. The memory savings occur because instances do not use a dynamic dictionary (`__dict__`) for attribute storage; instead, they use a fixed-sized array, which is less memory-intensive than a dictionary. Additionally, accessing attributes stored in `__slots__` follows a more direct memory access pattern, resulting in less indirection compared to a dictionary. Moreover, using `__slots__` ensures that instances cannot access attributes not listed in it.

It's worth mentioning that dynamic attribute assignment is still possible when using slots, although some of the memory size benefits are lost. This can be achieved by including `"__dict__"` in the `__slots__` definition, allowing for dynamic attributes in addition to those specified. The use of slots requires careful consideration due to the changes in class behavior it introduces.

More information about __slots__ can be found [here](https://stackoverflow.com/questions/472000/usage-of-slots).  

It is also possible to pass dunder methods dynamically (as seen above) to classes created dynamically:

In [45]:
def init(self):
    self.x = 5
    self.y = "A"


MyClass = type("MyClass", (), {"__init__": init})

my_instance = MyClass()
print(f"{my_instance.x} and {my_instance.y}")

5 and A


Another pattern that could be useful is to create a class factory to generate classes automatically:  

In [47]:
def class_builder(cls_name="MyClass"):
    # NOTE: it is possible to add a arg that map base classes
    def init(self):
        self.x = 25
        self.y = "B"

    return type(cls_name, (), {"__init__": init})


MyClass = class_builder()
my_instance = MyClass()
print(f"{my_instance.x} and {my_instance.y}")

25 and B


This type of pattern is useful for serialization and deserialiaztion of objects. The use of this type of pattern in general also follows the recommendations mentioned previously, spetially in situations where the data is known only at runtime, as mentioned [here](https://ctskennerton.github.io/2021/11/24/dynamic-types-in-python./).

Another situation that is useful being aware of is the possibility for passing default arguments to a subclass created dynamically:

In [2]:
class BaseClass:
    def __init_subclass__(cls, my_name):
        print(f"Subclass created and my name is {my_name}")
        # super().__init_subclass__() Needed only in case of multiple inheritance


MyDynamicSubClass = type("MyDynamicSubClass", (BaseClass,), {}, my_name="John")

Subclass created and my name is John


When is this pattern useful?

 - Runtime Configuration: When specific information is only available at runtime, allowing for dynamic class creation and customization.
- Validation and Registration: For implementing checks and registering instances to ensure data integrity and uniqueness.
- Default Data Provision: To pre-populate subclasses with relevant data based on their specific type or context.  

Further information about `__init_subclass__` can be found [here](https://stackoverflow.com/questions/63473901/python-dynamically-create-class-while-providing-arguments-to-init-subclass).

Another pattern useful for dynamic class creation from containers, like JSON, is to use a loop:

In [42]:
my_json = {
    "stuff": [
        {"name": "burger", "aka": ["cheeseburger", "hamburger"]},
        {"name": "fries", "aka": ["french fries", "potatoes"]},
        {
            "name": "meal",
            "items": [
                {"name": "burger", "value": "<burger>"},
                {"name": "fries", "value": "<fries>"},
            ],
        },
    ]
}


class MenuItem:

    def __init__(self, **kwargs) -> None:
        for key, value in kwargs.items():
            setattr(self, key, value)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__} with attributes {self.__dict__}"


for d in my_json["stuff"]:
    name = d.pop("name")
    NewClass = type(name, (MenuItem,), d)
    print(NewClass(**d))

burger with attributes {'aka': ['cheeseburger', 'hamburger']}
fries with attributes {'aka': ['french fries', 'potatoes']}
meal with attributes {'items': [{'name': 'burger', 'value': '<burger>'}, {'name': 'fries', 'value': '<fries>'}]}


In some cases, type hints may be required during dynamic class creation. In sistuations like this the best approach is to leverage the use of duncer methods in association with the `type` function:

In [3]:
type_hints = {"some_field": str}
MyClass = type(
    "MyClass",
    (),
    {
        "__doc__": "Class created dynamically",
        "__annotations__": type_hints,
        "some_field": "Some string",
    },
)
new_instace = MyClass()
new_instace.some_field, new_instace.__annotations__

('Some string', {'some_field': str})

Beside the `type` function, Python also has as an alternative to class creation the `types.new_class` function. The class creation process through `types.new_class` is signficantly more flexible when compared to the base `type` as it provides a mechanism for directly dealing with metaclasses and also allows a `exec_body` to be executed in the namespace of the class being created. More about the use of this approach can be found [here](https://docs.python.org/3/library/types.html#dynamic-type-creation).

### Value
The value of a object can change or not. This capabilitie defines if a object is mutable or immutable. But immutability is a tricky concept because a immutable object can have a reference to a mutable one. So, in this scenario, the value of a immutable object `can change` when the mutable object its referes to change. In this sense immutability is different from having a unchangeable value. Altough, the `id` of the object do not tend to change, at least in general.

In [59]:
num = 256
lst = [1, 2, num]
tpl = (1, 2, lst)
print(tpl)
print(id(num))
print(id(lst))
print(id(tpl))
num = (5545, "my tuple")
lst[2] = num
print(tpl)
print(id(num))
print(id(lst))
print(id(tpl))

(1, 2, [1, 2, 256])
8893480
140518044983424
140518044832640
(1, 2, [1, 2, (5545, 'my tuple')])
140518079995392
140518044983424
140518044832640


In the above code, the `id`s of lst and tpl did not changed because when a object is created in Python its id remains the same even after altering the value it is refering to.  
One other important thing is that, for immutable types, operations that compute a new value may actually return a reference to any existing object `a = 1; b = 1` where a and b may referes to the same object, but in `c = []; d = []` is guaranteed that c and d refers to different objects.  

### Data Structures
We discussed about the fundamental rules that govern Python objects. A logical next step in learning programming is to understand how these building blocks and rules can be leveraged to create larger structures.
Built-in Python sequences can be categorized as Container Sequences and Flat Sequences. The difference between these two is that the former can hold references to items of different types while the latter is a single object that can only store values of the same type in its memory space.  
In general, Flat Sequences are more compact, although they are limited to holding only primitive types.  
Every python object has header with metadata. For example, the a `float` has these fields:   
- on_refcnt: the object's reference count   
- ob_type: a pointer to the object's type   
- ob_val: a C double holding the value of the float  
 
And each one of these fields takes 8 bytes. For this reason, an array of floats is more compact than a tuple: the array is a single object holding the raw values of its items, while the tuple consists of several objects (the tuple itself and each object contained in it).  

Let's deepen our understanding of this difference a little more with the code below:

In [ ]:
import sys
import array
from pympler import asizeof

print("The size of a tuple:")

print(f"getsizeof: {sys.getsizeof(tuple([1.5, 2.1, 3.8, 4.8, 8.6, 4.9, 3.9]))}")
print(f"asizeof: {asizeof.asizeof(tuple([1.5, 2.1, 3.8, 4.8, 8.6, 4.9, 3.9]))}")

print(50 * "_")
print(50 * " ")

print("The size of an array:")

print(f'getsizeof: {sys.getsizeof(array.array("d", [1.5, 2.1, 3.8, 4.8, 8.6, 4.9, 3.9]))}')
print(f'asizeof: {asizeof.asizeof(array.array("d", [1.5, 2.1, 3.8, 4.8, 8.6, 4.9, 3.9]))}')

The size of a tuple:
getsizeof: 96
asizeof: 264
__________________________________________________
                                                  
The size of an array:
getsizeof: 136
asizeof: 136


We can see that an array occupies significantly less space than a tuple because it holds value directly rather than references, as mentioned previously.  

NOTE: Let's understand the difference between the two methods to measure the size of objects in Python.  

In the code above we used two methods of calculating the size of a python object. `getsizeof` returns the size of the object passed to itself, whereas `asizeof` returns the size of the object passed to it but also the objects referenced within the complex object instance.   

In general, `getsizeof` only reports the raw memory allocated (the header and its immediate contents) for the list's data and it is based on the interpreter's internal calculations.  
Meanwhile, `asizeof` calculate this by recursively adding up the sizes of all objects after traversing the object's contents and examining the object attributes. `asizeof` also avoids calculating the size of the same object twice, if it is referenced more than once, by tracking of the objects IDs that it already visited by using a `set` or a similar mechanism that allows it to recognize already-measured objects.  

It is important to note that it uses `getsizeof` to measure the size of the objects encountered and also accounts for the extra space allocated by container objects since some Python objects have over-allocation to optimize append operations, furthermore, it can detect1 shared objects, such as objects that are instances of classes with `__slots__` or some interned objects like strings.

Fonts:  
[One](hhttps://stackoverflow.com/questions/11301295/measure-object-size-accurately-in-python-sys-getsizeof-not-functioning)  
[Two](https://stackoverflow.com/questions/34787327/pympler-asizeof-vs-sys-getsizeof)  

Now, we will talk more about the most fundamental sequence types in Python.

### Lists, Generators, Tuples, Sets and, Dictionaries

#### Lists
Python has some commonly used built-in container types that are very handy and have a wide variety of use cases. The first of them, and perhaps the most used, is the `list`. Lists are present in pretty much everything Python touches due to their versatility. A notable example is list comprehensions (listcomp), which can be used for a wide range of situations. Two common patterns for using list comprehensions are replacing map and filter, and for Cartesian products, as demonstrated below. Using list comprehensions is a neat approach to reducing code complexity and can also be very useful when dealing with sorted lists.

In [ ]:
# Cartesian product
species = ["Urophylla", "Camaldulensis", "Citriodora", "Globulus"]
models = [80, 90, 89]

print("Cartesian Product:")
print([(specie, model) for specie in species for model in models])
print("")

# Sorted Cartesian Product
print("Sorted Cartesian Product:")
print(sorted(
    [(specie, model) for specie in species for model in models],
    key=lambda model: model[1],
    reverse=True,
))
print("")

print("Filtered listcomp:")
print([specie for specie in species if specie not in ["Urophylla", "Camaldulensis"]])

Cartesian Product:
[('Urophylla', 80), ('Urophylla', 90), ('Urophylla', 89), ('Camaldulensis', 80), ('Camaldulensis', 90), ('Camaldulensis', 89), ('Citriodora', 80), ('Citriodora', 90), ('Citriodora', 89), ('Globulus', 80), ('Globulus', 90), ('Globulus', 89)]

Sorted Cartesian Product:
[('Urophylla', 90), ('Camaldulensis', 90), ('Citriodora', 90), ('Globulus', 90), ('Urophylla', 89), ('Camaldulensis', 89), ('Citriodora', 89), ('Globulus', 89), ('Urophylla', 80), ('Camaldulensis', 80), ('Citriodora', 80), ('Globulus', 80)]

Filtered listcomp:
['Citriodora', 'Globulus']


Before proceeding with the discussion about other ways of creating sequences, let's talk a little about the Walrus Operator `:=`. This operator allows us to combine assignment and expression evaluation into a single operation. An interesting result of this is the possibility of avoiding code duplication, as shown below, which was common before the introduction of this operator:

In [3]:
my_list = [5, 6 , 8, 6]
if (size := len(my_list)) > 3:
    print(f"List size is {size}")

List size is 4


In [6]:
[[y := x + 1, x * y] for x in range(5)]

[[1, 0], [2, 2], [3, 6], [4, 12], [5, 20]]

#### Generators
Generator Expressions (genexps) are an alternative to lists and can be used to initialize other types of sequences. The use of genexps provides a significant advantage when the goal is to reduce memory usage. The approach that generators use is fairly simple, yet very powerful: they do not produce any values until requested. In other words, a generator expression does not return the collection of data upon its creation. The "forcing" of item production occurs through features like loops, collection construction and calls to `next` on the generator object. However, it should be noted that once the generator object is used, its contents are no longer available, and the error `StopIteration` is raised when all its items have been consumed.  



In [21]:
species = ["Urophylla", "Camaldulensis", "Citriodora"]
models = [80, 90, 89]

model_ranking = (
    f"{specie} yields: {model}" for specie in species for model in models
)
# We see that this object is a generator object
print(model_ranking)

# Consuming the genexp
for i in model_ranking:
    print(i)

# Creating a list and a set from a generator expression
list_from_ge = list((f"{specie} yields: {model}" for specie in species for model in models))
set_from_ge = set((f"{specie} yields: {model}" for specie in species for model in models))


print(list_from_ge)
print(set_from_ge)
# A error arrisen when trying to use a generator object after the consuption of its items
# next(model_ranking) raises StopIteration error

<generator object <genexpr> at 0x7f8eaef617e0>
Urophylla yields: 80
Urophylla yields: 90
Urophylla yields: 89
Camaldulensis yields: 80
Camaldulensis yields: 90
Camaldulensis yields: 89
Citriodora yields: 80
Citriodora yields: 90
Citriodora yields: 89
['Urophylla yields: 80', 'Urophylla yields: 90', 'Urophylla yields: 89', 'Camaldulensis yields: 80', 'Camaldulensis yields: 90', 'Camaldulensis yields: 89', 'Citriodora yields: 80', 'Citriodora yields: 90', 'Citriodora yields: 89']
{'Urophylla yields: 80', 'Camaldulensis yields: 90', 'Citriodora yields: 89', 'Camaldulensis yields: 89', 'Citriodora yields: 90', 'Citriodora yields: 80', 'Urophylla yields: 89', 'Camaldulensis yields: 80', 'Urophylla yields: 90'}


#### Tuples
Tuples are quite similar to lists, so much so that people sometimes tend to call tuples "immutable lists". This does not do justice to tuples as they are also records without field names (a concept similar to tuple structs in Rust). Before we dive into tuples' superpowers, let's understand how they can be used [to store items more effectively than lists](https://stackoverflow.com/questions/68630/are-tuples-more-efficient-than-lists-in-python).  

Tuples tend to be more efficient than lists in some aspects:

1. Tuples can be constant folded: In Computer Science, constant folding refers to the process of recognizing and evaluating constant expressions at compile time rather than computing them at runtime. In Python, this process is used to recognize when a tuple contains only constants and optimize it by precomputing the tuple during compilation. In the example below, we can see that more steps are necessary to build a list because Python treats the entire tuple as a constant when it contains only constants. In this scenario, the immutable nature of tuples is leveraged to improve their creation performance:

In [7]:
from dis import dis
dis(compile("(10, 'abc')", "", "eval"))

print(60 * "-")

dis(compile("[10, 'abc']", '', 'eval'))

  0           RESUME                   0

  1           RETURN_CONST             0 ((10, 'abc'))
------------------------------------------------------------
  0           RESUME                   0

  1           LOAD_CONST               0 (10)
              LOAD_CONST               1 ('abc')
              BUILD_LIST               2
              RETURN_VALUE


2. Tuples are not copied: Since tuples are immutable structures, they are not copied upon being passed as arguments to other tuples. In this case, Python recognizes that it is safe to reuse a tuple instead of creating a new one. Hence, calling a tuple on a tuple is faster and more efficient than calling a list on a list.

In [8]:
a = (10, 20, 30)
b = tuple(a)
a is b

True

In [9]:
a = [10, 20, 30]
b = list(a)
a is b

False

3. Tuples do not overallocate: Since tuples are immutable, their sizes are fixed. Therefore, it is not necessary to allocate extra memory to account for future item additions through `append()` calls. NOTE: although true, this is not particularly significant at the present moment. Looking at the results below, it is possible to observe that the differences are not drastic as they were once in previous Python versions.

In [10]:
from pympler import asizeof

asizeof.asizeof(tuple(iter(range(50000)))), asizeof.asizeof(list(iter(range(50000))))

(2000040, 2000056)

4. Tuples refer directly to their elements: Tuples do not have the indirection layer found in lists, where they point to an array of pointers. Instead, they incorporate their references directly into the tuple object. Note that this holds slightly for indexing, but not as much for unpacking, and holds significantly for assignment.

$python -m timeit -s 'a = (10, 20, 30)' 'a[1]'
50000000 loops, best of 5: 4.85 nsec per loop
$python -m timeit -s 'a = [10, 20, 30]' 'a[1]'
50000000 loops, best of 5: 4.6 nsec per loop

$python -m timeit -s 'a = (10, 20, 30)' 'x, y, z = a'
50000000 loops, best of 5: 5.43 nsec per loop
$python -m timeit -s 'a = [10, 20, 30]' 'x, y, z = a'
50000000 loops, best of 5: 5.66 nsec per loop

$python -m timeit "x=(1,2,3,4,5,6,7,8)"
50000000 loops, best of 5: 3.78 nsec per loop
$python -m timeit "x=[1,2,3,4,5,6,7,8]"
20000000 loops, best of 5: 16.4 nsec per loop

Let's go back and explore how tuples can be used as records without field names. Tuples typically have a fixed number of items and their order matters. This makes them especially useful in situations where we can use unpacking to retrieve items from a tuple, and avoid the overhead of creating a class just to name the fields:

In [14]:
flight_order = [("São Paulo", "5:30"), ("Bahia", "12:45"), ("Pará", "14:50")]

for city, depart_time in flight_order:
    print(f"Depart from {city}, {depart_time} o'clock")

city, depart_time = flight_order[0]
city, depart_time

Depart from São Paulo, 5:30 o'clock
Depart from Bahia, 12:45 o'clock
Depart from Pará, 14:50 o'clock


('São Paulo', '5:30')